In [2]:
ifile = "st23_electron_520_10000eve.root" 

In [3]:
# import libraries
import uproot3 as uproot
import uproot as uproot4
import numpy as np
import numba
import awkward as ak
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import                                                                                                                  
import mplhep as hep
plt.style.use(hep.style.ROOT)
import os,sys
import pandas as pd
from scipy.stats import halfnorm
from sklearn.cluster import KMeans
import random
from scipy.stats import binned_statistic
from scipy.stats import binned_statistic_2d
from scipy.optimize import curve_fit
from scipy.stats import crystalball
from lmfit import Model
import copy
import math
from tqdm import tqdm

#import ROOT
#from coffea import hist

In [4]:
# Process Data
def getData(fname="", procName="Events"):
    kn_dict = uproot.open(fname)["Events"].arrays()
    kn_dict_ak1 = {name.decode(): ak.from_awkward0(array) for name, array in kn_dict.items()}
    kn_events = ak.zip({"Tracks":ak.zip({
                                        "n_tracks":   kn_dict_ak1["n_tracks"],
                                        "truthtrack_x_st3":   kn_dict_ak1["truthtrack_x_st3"],
                                        "truthtrack_y_st3":   kn_dict_ak1["truthtrack_y_st3"],
                                        "truthtrack_z_st3":   kn_dict_ak1["truthtrack_z_st3"],
                                        "truthtrack_x_st1":   kn_dict_ak1["truthtrack_x_st1"],
                                        "truthtrack_y_st1":   kn_dict_ak1["truthtrack_y_st1"],
                                        "truthtrack_z_st1":   kn_dict_ak1["truthtrack_z_st1"],
                                        "truthtrack_x_vtx":   kn_dict_ak1["truthtrack_x_vtx"],
                                        "truthtrack_y_vtx":   kn_dict_ak1["truthtrack_y_vtx"],
                                        "truthtrack_z_vtx":   kn_dict_ak1["truthtrack_z_vtx"],
                                      }),
                        "RecoTracks":ak.zip({
                                      "n_tracks":   kn_dict_ak1["n_tracks"],
                                        "track_x_HCAL":   kn_dict_ak1["track_x_CAL"],
                                        "track_y_CAL":   kn_dict_ak1["track_y_CAL"],
                                        "track_pz":      kn_dict_ak1["track_pz_st3"],
                                        "track_px":      kn_dict_ak1["track_px_st3"],
                                        "track_py":      kn_dict_ak1["track_py_st3"],
                                        "track_x_st3":      kn_dict_ak1["track_x_st3"],
                                        "track_y_st3":      kn_dict_ak1["track_y_st3"],
                                        "track_z_st3":      kn_dict_ak1["track_z_st3"]
                                      }),
                        "Electrons":ak.zip({
                                            "ge":      kn_dict_ak1["ge"],
                                            "gvx":      kn_dict_ak1["gvx"],
                                            "gvy":      kn_dict_ak1["gvy"],
                                            "gvz":      kn_dict_ak1["gvz"],
                                            "gpx":      kn_dict_ak1["gpx"],
                                            "gpy":      kn_dict_ak1["gpy"],
                                            "gpz":      kn_dict_ak1["gpz"],
                                        }),
                        "Hits":ak.zip({
                                      "detID":   kn_dict_ak1["hit_detID"],
                                      "edep":    kn_dict_ak1["hit_edep"],
                                      "elmID":   kn_dict_ak1["hit_elmID"],
                                      "truthx":  kn_dict_ak1["hit_truthx"],
                                      "truthy":  kn_dict_ak1["hit_truthy"],
                                      "truthz":  kn_dict_ak1["hit_truthz"],
                                      "hit_pos":  kn_dict_ak1["hit_pos"],
                                      }),
                        "Cal_Position":ak.zip({
                                            "track_x_CAL":      kn_dict_ak1["track_x_CAL"],
                                            "track_y_CAL":     kn_dict_ak1["track_y_CAL"],
                                        }),
                        "St23_Tracklet":ak.zip({
                                        "n_tracks":   kn_dict_ak1["n_tracks"],
                                        "st23tracklet_nhits_st1":   kn_dict_ak1["st23tracklet_nhits_st1"],
                                        "st23tracklet_nhits_st2":   kn_dict_ak1["st23tracklet_nhits_st2"],
                                        "st23tracklet_nhits_st3":   kn_dict_ak1["st23tracklet_nhits_st3"],
                                        "st23tracklet_px_st1":   kn_dict_ak1["st23tracklet_px_st1"],
                                        "st23tracklet_px_st3":   kn_dict_ak1["st23tracklet_px_st3"],
                                        "st23tracklet_py_st1":   kn_dict_ak1["st23tracklet_py_st1"],
                                        "st23tracklet_py_st3":   kn_dict_ak1["st23tracklet_py_st3"],
                                        "st23tracklet_pz_st1":   kn_dict_ak1["st23tracklet_pz_st1"],
                                        "st23tracklet_pz_st3":   kn_dict_ak1["st23tracklet_pz_st3"],
                                        "st23tracklet_x_st1":   kn_dict_ak1["st23tracklet_x_st1"],
                                        "st23tracklet_x_st3":   kn_dict_ak1["st23tracklet_x_st3"],
                                        "st23tracklet_x_CAL":   kn_dict_ak1["st23tracklet_x_CAL"],
                                        "st23tracklet_y_st1":   kn_dict_ak1["st23tracklet_y_st1"],
                                        "st23tracklet_y_st3":   kn_dict_ak1["st23tracklet_y_st3"],
                                        "st23tracklet_y_CAL":   kn_dict_ak1["st23tracklet_y_CAL"],
                                        "st23tracklet_z_st1":   kn_dict_ak1["st23tracklet_z_st1"],
                                        "st23tracklet_z_st3":   kn_dict_ak1["st23tracklet_z_st3"],  
                                        }),
                        "coordinates":ak.zip({
                                        "sx_ecal":   kn_dict_ak1["sx_ecal"],
                                        "sy_ecal":   kn_dict_ak1["sy_ecal"],
                                        "sz_ecal":   kn_dict_ak1["sz_ecal"],
                            
                                        }),
                            
                    
                       }, depth_limit=1)
    return kn_events

In [5]:
def emcal_simhit_selection(arr):
    mask = (arr.detID == 100)
    return mask

def emcal_simhit_selection_energy(arr, e):
    mask = (arr.edep >= e)
    return mask

def h2_selection(arr):
    mask = (arr.detID >= 35) & (arr.detID <= 38)
    return mask

def st2_selection(arr):
    mask = (arr.detID >= 13) & (arr.detID <= 18)
    return mask

def st3_selection(arr):
    mask = (arr.detID >= 19) & (arr.detID <= 30)
    return mask

def h4_selection(arr):
    mask = (arr.detID >= 41) & (arr.detID <= 46)
    return mask

In [6]:
#importing data from file
kn_events = getData(ifile,"Events")

In [7]:
class EMCal(object):
    """
    define the EMCal Geometry
    """
    def __init__(self):
        self.ntowersx=72
        self.ntowersy=36
        self.sizex=5.53 # in cm
        self.sizey=5.53 # in cm

        self.ecalx=[-200,200] #size in cm
        self.ecaly=[-100,100]
        self.binsx=self.ecalx[1]- self.ecalx[0]
        self.binsy=self.ecaly[1]- self.ecaly[0]
        sfc = 0.1146337964120158

In [8]:
def emcal_byevent(evtNum):
    emin=0.0005
    algo=0
    dq_events = getData(ifile,"Truth")
    dq_hits = dq_events[evtNum]["Hits"]
    dq_electrons = dq_events[evtNum]["Electrons"]
    ge = ak.to_list(dq_electrons.ge)
    # select emcal hits
    emcal_mask = emcal_simhit_selection(dq_hits)
    emcal_hits = dq_hits[emcal_mask]
    emcal_energy_mask = emcal_simhit_selection_energy(emcal_hits, emin)
    emcal_hits = emcal_hits[emcal_energy_mask]
    
    emcal = EMCal()
    emcal_towerx = emcal_hits.elmID//emcal.ntowersy
    emcal_towery = emcal_hits.elmID%emcal.ntowersy
    emcal_truthx = emcal_hits.truthx
    emcal_truthy = emcal_hits.truthy
    emcal_x = emcal.ecalx[0]+emcal_towerx*emcal.sizex
    emcal_y = emcal.ecaly[0]+emcal_towery*emcal.sizey
    emcal_edep = emcal_hits.edep
    return emcal_x, emcal_y, emcal_edep

In [9]:
#Calculating the coorinates, and energies, getting those basic info
x_pos = []#designed to be 2D
y_pos = []#designed to be 2D
track_energy = []#designed to be 2D
failed_energy = []#store noise energy values

for i in range(len(kn_events[:]["Hits"].edep)):
    output=emcal_byevent(i)
    event_x = output[0]
    event_y = output[1]
    event_energy = output[2]
    
    track_energy.append(event_energy)
    x_pos.append(event_x)
    y_pos.append(event_y)

In [10]:
#calculate x_avg, y_avg and e_tot by event for the whole n-tuple
x_eve_avg = []#1D, 10,000
y_eve_avg = []#1D, 10,000
energy_tot = []#1D, 10,000

for i in range(len(track_energy)):
    event_x_avg=sum(x_pos[i])/(len(x_pos[i]))
    event_y_avg=sum(y_pos[i])/(len(y_pos[i]))
    event_energy_tot=sum(track_energy[i])
    x_eve_avg.append(event_x_avg)
    y_eve_avg.append(event_y_avg)
    energy_tot.append(event_energy_tot)

In [11]:
#calculate x_bar and y_bar for wew by event
x_bar_eve = []#1D, 10,000
y_bar_eve = []#1D, 10,000
for i in range(len(track_energy)):
    x_bar=0
    y_bar=0
    for j in range(len(track_energy[i])):
        x_bar+=track_energy[i][j]*x_pos[i][j]
        y_bar+=track_energy[i][j]*y_pos[i][j]
    x_bar_eve.append(x_bar/energy_tot[i])
    y_bar_eve.append(y_bar/energy_tot[i])

In [50]:
print("average of x coordinates ", sum(x_eve_avg)/len(x_eve_avg))
print("average of y coordinates ", sum(y_eve_avg)/len(y_eve_avg))
print(len(x_bar_eve))

average of x coordinates  -2.5743401752667885
average of y coordinates  6.739956857664715
10000


In [39]:
wew_x = []#1D, 10,000
wew_y = []#1D, 10,000
for i in range(len(track_energy)):
    sum_x=0
    sum_y=0
    for j in range(len(track_energy[i])):
        sum_x+=track_energy[i][j]*pow((x_pos[i][j]-x_bar_eve[i]),2)
        sum_y+=track_energy[i][j]*pow((y_pos[i][j]-y_bar_eve[i]),2)
    wew_x.append(math.sqrt(sum_x/energy_tot[i]))
    wew_y.append(math.sqrt(sum_y/energy_tot[i]))

In [40]:
n=0
for i in range(len(wew_x)):
    if wew_x[i]>n:
        n=wew_x[i]
print(n) 

93.68618339508234


In [41]:
print(sum(wew_x)/len(wew_x))
print(sum(wew_y)/len(wew_y))


3.2737503980063423
11.722506864113276


In [42]:
wew_x_int= []
wew_y_int =[]
for i in range(len(wew_x)):
    wew_x_int.append(round(wew_x[i]))
    wew_y_int.append(round(wew_y[i]))

In [43]:
wew_x_stat=[0]*100
wew_y_stat=[0]*100
for i in range(len(wew_x)):
    wew_x_stat[wew_x_int[i]]+=1
    wew_y_stat[wew_y_int[i]]+=1

In [45]:
print(wew_y_stat)

[0, 32, 1120, 457, 342, 814, 1134, 556, 273, 199, 389, 711, 605, 249, 122, 173, 331, 535, 225, 109, 84, 129, 258, 217, 63, 40, 49, 89, 147, 67, 21, 26, 35, 56, 57, 22, 13, 9, 17, 47, 13, 14, 4, 8, 17, 17, 2, 4, 7, 16, 17, 6, 3, 2, 6, 5, 9, 4, 2, 1, 3, 4, 5, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [47]:
print(wew_x_stat)

[0, 465, 5940, 2061, 365, 236, 199, 95, 96, 73, 47, 58, 30, 40, 35, 15, 22, 22, 17, 20, 13, 15, 13, 8, 9, 11, 8, 11, 9, 3, 9, 7, 3, 6, 5, 3, 4, 1, 3, 5, 1, 1, 1, 0, 3, 2, 1, 0, 0, 2, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
